# COGS 108 Final Project

## Group Members: 
- A14178715
- A11475687
- A12594395
- A11729103

### Introduction and Background:

What would it look like if we visualized a predicted sea level rise for the future? How accurate can we get based on a large dataset of the sea level rising over a long period of time?

Well, we should see a fairly accurate model of future sea level, represented by a map of the coastal regions of California flooded. This model will get less and less accurate the further we go into the future. To start, we need data. Here is a good one:

ftp://podaac.jpl.nasa.gov/allData/merged_alt/L2/TP_J1_OSTM/global_mean_sea_level/GMSL_TPJAOS_V4_199209_201702.txt

This is NASA's data on calculating the sea level increase with satellites. The link worked a while back, but no longer works, so we downloaded the data while it was available and have included it in our repository.


### Data Description:

NASA's description of the data:

"Global Mean Sea Level Data
This file contains Global Mean Sea Level (GMSL) variations computed at the NASA Goddard Space Flight Center under the 
auspices of the NASA MEaSUREs program. The GMSL was generated using the Integrated Multi-Mission Ocean Altimeter Data for Climate Research (http://podaac.jpl.nasa.gov/dataset/MERGED_TP_J1_OSTM_OST_ALL_V4). It combines Sea Surface Heights from TOPEX/Poseidon, Jason-1, OSTM/Jason-2, and Jason-3 to a common terrestrial reference frame with all inter-mission biases, range and geophysical corrections applied and placed onto a georeferenced orbit. This creates a consistent data record throughout time, regardless of the instrument used."

In [1]:
# imports 
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

import csv

txt_file = r"nasa_data_noheader.txt"
csv_file = r"nasa_data_formatted.csv"

# use 'with' if the program isn't going to immediately terminate
# so you don't leave files open
# the 'b' is necessary on Windows
# it prevents \x1a, Ctrl-z, from ending the stream prematurely
# and also stops Python converting to / from different line terminators
# On other platforms, it has no effect
in_txt = csv.reader(open(txt_file, "rb"), delimiter = '\t')
out_csv = csv.writer(open(csv_file, 'wb'))

out_csv.writerows(in_txt)
out_csv

Error: iterator should return strings, not bytes (did you open the file in text mode?)

### Data Cleaning and Pre-processing

In [ ]:
# read in the data, nasa_data.txt as csv


In [ ]:
# clean data (remove weird header or load in nasa_data_noheader.txt)


### Data Visualization

In [ ]:
# plot data using plt.something()


In [ ]:
# predict future sea level rise for the next 100 years

In [ ]:
# show what this prediction looks like in a google map

### Conclusions and Discussion:

the end